In [19]:
# verifying pickle datas

import pandas as pd
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

df = pd.read_pickle(os.path.join(project_root, 'pickles', '1min', 'all_agg_open_interest.pkl'))
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
    print(df[list(df.keys())[5]].head())

df = pd.read_pickle(os.path.join(project_root, 'pickles', '10min', 'all_agg_open_interest.pkl'))
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
    print(df[list(df.keys())[5]].head())
    print(list(df.keys())[5])

   Side              DeliveryStart                DeliveryEnd  Price  Volume BlockVolume               Trading_time
0   BUY  2021-06-25 22:00:00+00:00  2021-06-25 22:15:00+00:00   63.0     2.5         [0]  2021-06-25 13:05:00+00:00
1  SELL  2021-06-25 22:00:00+00:00  2021-06-25 22:15:00+00:00  500.0     1.0      [0, 0]  2021-06-25 13:05:00+00:00
2   BUY  2021-06-25 22:15:00+00:00  2021-06-25 22:30:00+00:00   63.0     2.5         [0]  2021-06-25 13:05:00+00:00
3  SELL  2021-06-25 22:15:00+00:00  2021-06-25 22:30:00+00:00  500.0     1.0      [0, 0]  2021-06-25 13:05:00+00:00
4   BUY  2021-06-25 22:30:00+00:00  2021-06-25 22:45:00+00:00   63.0     2.5         [0]  2021-06-25 13:05:00+00:00
    Side             DeliveryStart               DeliveryEnd   Price  Volume BlockVolume              Trading_time      Unique_Data_ID
0    BUY 2021-06-25 22:00:00+00:00 2021-06-25 23:00:00+00:00   13.90    20.0      [0, 0] 2021-06-25 13:50:00+00:00   20210625-135000.0
1    BUY 2021-06-25 22:00:00+00:00

In [1]:
import pandas as pd
from pandas import DatetimeIndex
import numpy as np

def date_range(start, end, freq):
    if start == end:
        return DatetimeIndex([])
    else:
        return pd.date_range(start=start, end=end, freq=freq, inclusive='left')

# TODO: read from pickle instead

df = pd.read_csv('../20210625-130000_to_20210626-225000_full_transaction_book.csv')
expanded_rows = []

for _, row in df.iterrows():
    row['TransactionTime'] = pd.to_datetime(row['TransactionTime'], errors='coerce')
    row['DeliveryStart'] = pd.to_datetime(row['DeliveryStart'], errors='coerce')
    row['DeliveryEnd'] = pd.to_datetime(row['DeliveryEnd'], errors='coerce')

    for hour in date_range(start=row['DeliveryStart'].floor('h'), end=row['DeliveryEnd'].floor('h'), freq='h'):
        new_row = row.copy()
        new_row['DeliveryHour'] = hour
        expanded_rows.append(new_row)

In [2]:
RESOLUTION = '10min'

hourly_df = pd.DataFrame(expanded_rows)
hourly_df = hourly_df.sort_values('DeliveryHour').reset_index(drop=True)

def agg_func(group):
    min_price = group['Price'].min()
    max_price = group['Price'].max()
    vwap = np.nan if group['Volume'].sum() == 0 else np.average(group['Price'], weights=group['Volume'])
    total_volume = group['Volume'].sum()

    group_sorted = group.sort_values('TransactionTime')
    price_diff = group_sorted['Price'].diff()

    zero_diff_volume = group_sorted.loc[price_diff == 0, 'Volume'].sum()
    up_volume = group_sorted.loc[price_diff > 0, 'Volume'].sum()
    down_volume = group_sorted.loc[price_diff < 0, 'Volume'].sum()

    return pd.Series({'MinPrice': min_price, 'MaxPrice': max_price, 'VWAP': vwap, 'TotalVolume': total_volume, 'UpVolume': up_volume, 'DownVolume': down_volume, 'ZeroDiffVolume': zero_diff_volume})

resampled_df = (
    hourly_df
    .set_index('TransactionTime')
    .groupby('DeliveryHour')
    .resample(RESOLUTION, include_groups=False)
    .apply(agg_func)
    .reset_index()
)

unique_hours = hourly_df['DeliveryHour'].drop_duplicates().sort_values().tolist()

# print(hourly_df.head(20).to_string())
# print(resampled_df.head(20).to_string())

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.frequencies import to_offset

def plot(idx, additions=None, disable_volume_bubbles=False, disable_raw_price_line=False, disable_min_max_vwap=False, price_range=[None, None]):
    hour = unique_hours[idx]
    df_raw = hourly_df[hourly_df['DeliveryHour'] == hour].sort_values(['TransactionTime', 'OrderId'])
    
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3], vertical_spacing=0.2,
        subplot_titles=(f'Combined Price Plot for Delivery Hour {hour} (Aggregating per {RESOLUTION})', 'Up/Down Volume'),
        specs=[[{"secondary_y": True}], [{}]]  # Add secondary y-axis for the first graph
    )

    df_buy = df_raw[df_raw['Side'] == 'BUY']
    df_sell = df_raw[df_raw['Side'] == 'SELL']

    if not disable_volume_bubbles:
        fig.add_trace(go.Scatter(
            x=df_sell['TransactionTime'],
            y=df_sell['Price'],
            mode='markers',
            name='SELL',
            marker=dict(
                color='green',
                size=df_sell['Volume'] / df_raw['Volume'].max() * 100,
                opacity=0.5
            )
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=df_buy['TransactionTime'],
            y=df_buy['Price'],
            mode='markers',
            name='BUY',
            marker=dict(
                color='red',
                size=df_buy['Volume'] / df_raw['Volume'].max() * 100,
                opacity=0.5
            )
        ), row=1, col=1)

    if not disable_raw_price_line:
        fig.add_trace(go.Scatter(
            x=df_raw['TransactionTime'],
            y=df_raw['Price'],
            mode='markers+lines',
            name='Raw Price',
            line=dict(color='black', width=1),
            opacity=0.3,
            text=df_raw.apply(lambda r: f"OrderId: {r['OrderId']}<br>Side: {r['Side']}<br>Volume: {r['Volume']}<br>Price: {r['Price']}<br>ActionCode: {r['ActionCode']}<br>DeliveryStart: {r['DeliveryStart']}<br>DeliveryEnd: {r['DeliveryEnd']}", axis=1),
            hoverinfo='text+x+y'
        ), row=1, col=1)

    df_minmax = resampled_df[resampled_df['DeliveryHour'] == hour].sort_values('TransactionTime').copy()
    df_minmax['WindowEnd'] = df_minmax['TransactionTime'] + to_offset(RESOLUTION)

    if not disable_min_max_vwap:
        fig.add_trace(go.Scatter(
            x=df_minmax['WindowEnd'],
            y=df_minmax['MinPrice'],
            mode='lines',
            name='Min Price',
            line=dict(color='green', width=4),
            connectgaps=True
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=df_minmax['WindowEnd'],
            y=df_minmax['MaxPrice'],
            mode='lines',
            name='Max Price',
            line=dict(color='red', width=4),
            connectgaps=True
        ), row=1, col=1)
        fig.add_trace(go.Scatter(
            x=df_minmax['WindowEnd'],
            y=df_minmax['VWAP'],
            mode='lines',
            name='Volume-Weighted Average Price',
            line=dict(color='orange', width=2),
            connectgaps=True
        ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=df_minmax['WindowEnd'],
        y=df_minmax['UpVolume'],
        name='Up Volume',
        marker_color='limegreen',
        opacity=0.7
    ), row=2, col=1)
    fig.add_trace(go.Bar(
        x=df_minmax['WindowEnd'],
        y=df_minmax['DownVolume'],
        name='Down Volume',
        marker_color='crimson',
        opacity=0.7
    ), row=2, col=1)
    fig.add_trace(go.Bar(
        x=df_minmax['WindowEnd'],
        y=df_minmax['ZeroDiffVolume'],
        name='Zero Diff Volume',
        marker_color='gray',
        opacity=0.7
    ), row=2, col=1)

    for addition in (additions or []):
        fig.add_trace(addition, row=1, col=1)

    fig.update_layout(
        xaxis=dict(
            title='Time',
            tickformat='%H:%M',
            dtick=15*60*1000,  # 15 minutes in ms
            showgrid=True,
            gridcolor='lightgrey',
            minor=dict(dtick=5 * 60 * 1000, showgrid=True, gridcolor='whitesmoke')
        ),
        xaxis2=dict(  # Add secondary x-axis for time display
            title='Time (Secondary)',
            tickformat='%H:%M',
            dtick=15*60*1000,
            showgrid=True,
            gridcolor='lightgrey',
            minor=dict(dtick=5 * 60 * 1000, showgrid=True, gridcolor='whitesmoke')
        ),
        yaxis_title='Price',
        xaxis2_title='Time',
        yaxis2_title='Volume',
        barmode='relative',
        showlegend=True
    )

    if price_range[0] or price_range[1]:
        fig.update_yaxes(range=price_range, row=1, col=1)
    
    fig.show(renderer="browser")

plot(0)

In [4]:
import pickle

with open('../pickle/all_agg_open_interest.pkl', 'rb') as f:
    all_agg_open_interest = pickle.load(f)

    # type conversions
    for key in all_agg_open_interest.keys():
        lob = all_agg_open_interest[key]
        lob['DeliveryStart'] = pd.to_datetime(lob['DeliveryStart'])
        lob['DeliveryEnd'] = pd.to_datetime(lob['DeliveryEnd'])
        lob['Trading_time'] = pd.to_datetime(lob['Trading_time'])

In [5]:
def dual_thrust_signals(df, k1=0.5, k2=0.5, fill_missing=True):
    """
    Calculate Dual-Thrust breakout levels and signals for a given DataFrame.
    Assumes df is sorted by TransactionTime and contains 'MinPrice', 'MaxPrice', 'VWAP'.
    Handles missing values by forward-filling the last seen values.
    """
    prev_high = df['MaxPrice'].shift(1)
    prev_low = df['MinPrice'].shift(1)
    prev_close = df['VWAP'].shift(1)

    if fill_missing:
        prev_high = prev_high.ffill()
        prev_low = prev_low.ffill()
        prev_close = prev_close.ffill()

    range_ = (prev_high - prev_low)
    upper = prev_close + k1 * range_
    lower = prev_close - k2 * range_
    return upper, lower

In [6]:
from bisect import bisect_left

def backtest_dual_thrust(df, order_book, k1=0.5, k2=0.5, fill_missing=True, initial_balance=10000):
    """
    Backtest the Dual-Thrust strategy on the given DataFrame using limit order book for fills.
    Parameters:
        df (DataFrame): DataFrame containing 'TransactionTime', 'MinPrice', 'MaxPrice', 'VWAP'.
        k1, k2 (float): Dual-Thrust breakout parameters.
        initial_balance (float): Starting balance for the backtest.
        order_book (dict): Limit order book data mapping time to DataFrame.
    Returns:
        results (dict): Dictionary containing backtest results.
    """
    
    balance = initial_balance
    position = 0
    trades = []

    df['UpperBreakout'], df['LowerBreakout'] = dual_thrust_signals(df, k1=k1, k2=k2, fill_missing=fill_missing)
    order_book_times = sorted([pd.to_datetime(t).tz_localize('UTC') for t in order_book.keys()])
    
    for i, row in df.iterrows():
        if pd.isna(row['UpperBreakout']) or pd.isna(row['LowerBreakout']):
            continue

        current_time = row['TransactionTime']
        idx = bisect_left(order_book_times, current_time)
        if idx < len(order_book_times):
            nearest_time = order_book_times[idx]
        else:
            continue

        lob = order_book[nearest_time.strftime('%Y%m%d-%H%M%S')]
        best_bid = lob[lob['Side'] == 'BUY']['Price'].max()
        best_ask = lob[lob['Side'] == 'SELL']['Price'].min()

        # TODO: check volume and ensure sufficient liquidity

        if position == 0:
            if row['VWAP'] > row['UpperBreakout'] and best_ask <= row['VWAP']:
                position = balance / best_ask  # Buy at best ask
                balance = 0
                trades.append({'time': nearest_time, 'action': 'BUY', 'price': best_ask, 'position': position, 'balance': balance})
            elif row['VWAP'] < row['LowerBreakout'] and best_bid >= row['VWAP']:
                position = -balance / best_bid  # Short at best bid
                balance = 0
                trades.append({'time': nearest_time, 'action': 'SELL', 'price': best_bid, 'position': position, 'balance': balance})
        elif position > 0:
            if row['VWAP'] < row['LowerBreakout'] and best_bid >= row['VWAP']:
                balance = position * best_bid  # Sell at best bid
                trades.append({'time': nearest_time, 'action': 'SELL', 'price': best_bid, 'position': 0, 'balance': balance})
                position = 0
        elif position < 0:
            if row['VWAP'] > row['UpperBreakout'] and best_ask <= row['VWAP']:
                balance = -position * best_ask  # Cover at best ask
                trades.append({'time': nearest_time, 'action': 'BUY', 'price': best_ask, 'position': 0, 'balance': balance})
                position = 0

    # Final balance calculation
    if position > 0:
        balance += position * best_bid
        trades.append({'time': nearest_time, 'action': 'SELL', 'price': best_bid, 'position': 0, 'balance': balance})
    elif position < 0:
        balance += -position * best_ask
        trades.append({'time': nearest_time, 'action': 'BUY', 'price': best_ask, 'position': 0, 'balance': balance})

    return {
        'final_balance': balance,
        'trades': trades,
        'net_profit': balance - initial_balance
    }

In [7]:
initial_balance = 10000

all_results = {}

for hour in unique_hours:
    df_minmax_hour = resampled_df[resampled_df['DeliveryHour'] == hour].sort_values('TransactionTime').copy()
    result = backtest_dual_thrust(df_minmax_hour, k1=0.6, k2=0.4, initial_balance=initial_balance, order_book=all_agg_open_interest)
    all_results[hour] = result

net_profits = [result['net_profit'] for result in all_results.values()]

print("-" * 40)

for label, value, tIdx in [
    ("High Profit", np.max(net_profits), np.argmax(net_profits)),
    ("Low Profit", np.min(net_profits), np.argmin(net_profits)),
    ("25th Percentile", np.percentile(net_profits, 25), None),
    ("50th Percentile", np.percentile(net_profits, 50), None),
    ("75th Percentile", np.percentile(net_profits, 75), None),
]:
    print(f"{label}: {value} ({value / initial_balance * 100:.2f}%)" + (f" at {unique_hours[tIdx].strftime('%m-%d %H:%M')} (index {tIdx})" if tIdx is not None else ""))

print("-" * 40)

for hour, result in all_results.items():
    print(f"Hour: {hour.strftime('%m-%d %H:%M')} | Net Profit: {result['net_profit']}")
    # print(f"  Trades: {len(result['trades'])}")
    # for trade in result['trades']:
    #     print(f"    {trade}")

----------------------------------------
High Profit: 45032618.185246944 (450326.18%) at 06-26 21:00 (index 23)
Low Profit: -9969.929730929378 (-99.70%) at 06-26 15:00 (index 17)
25th Percentile: -6867.213053399592 (-68.67%)
50th Percentile: -3489.88238449194 (-34.90%)
75th Percentile: 3240.657167840413 (32.41%)
----------------------------------------
Hour: 06-25 22:00 | Net Profit: -4706.819549245717
Hour: 06-25 23:00 | Net Profit: -5558.482315513503
Hour: 06-26 00:00 | Net Profit: 3129.1605298818613
Hour: 06-26 01:00 | Net Profit: -5527.757108930325
Hour: 06-26 02:00 | Net Profit: 3575.147081716068
Hour: 06-26 03:00 | Net Profit: 34010.491845508455
Hour: 06-26 04:00 | Net Profit: -3122.0222035450124
Hour: 06-26 05:00 | Net Profit: -3857.742565438867
Hour: 06-26 06:00 | Net Profit: 69.8409596919173
Hour: 06-26 07:00 | Net Profit: 2949.3439686496804
Hour: 06-26 08:00 | Net Profit: -764.5048015788652
Hour: 06-26 09:00 | Net Profit: -4597.498704276954
Hour: 06-26 10:00 | Net Profit: 175

In [14]:
def plot_dual_thrust(idx=0, k1=0.5, k2=0.5, price_range=[None, None]):
    hour = unique_hours[idx]
    df_minmax = resampled_df[resampled_df['DeliveryHour'] == hour].sort_values('TransactionTime').copy()
    df_minmax['UpperBreakout'], df_minmax['LowerBreakout'] = dual_thrust_signals(df_minmax, k1=k1, k2=k2)
    df_minmax['RowIndex'] = range(len(df_minmax))

    df_minmax_offset = df_minmax.copy()
    df_minmax_offset['TransactionTime'] += pd.to_timedelta(RESOLUTION)
    df_minmax = pd.concat([df_minmax, df_minmax_offset]).sort_values(['TransactionTime', 'RowIndex'])

    trades = all_results[hour]['trades']
    buys = [trade for trade in trades if trade['action'] == 'BUY']
    sells = [trade for trade in trades if trade['action'] == 'SELL']

    plot(idx, [
        go.Scatter(
            x=df_minmax['TransactionTime'],
            y=df_minmax['UpperBreakout'],
            mode='markers+lines',
            name='Upper Breakout',
            line=dict(color='purple', width=2, dash='dash'),
            connectgaps=False
        ),
        go.Scatter(
            x=df_minmax['TransactionTime'],
            y=df_minmax['LowerBreakout'],
            mode='markers+lines',
            name='Lower Breakout',
            line=dict(color='brown', width=2, dash='dash'),
            connectgaps=False
        ),

        go.Scatter(
            x=[trade['time'] for trade in trades],
            y=[trade['price'] for trade in trades],
            mode='lines+markers',
            name='Trade Path',
            line=dict(color='navy', width=2, dash='dot'),
            marker=dict(size=6, color='navy', opacity=0.3)
        ),
        go.Scatter(
            x=[trade['time'] for trade in buys],
            y=[trade['price'] for trade in buys],
            mode='markers',
            name='Buys',
            marker=dict(color='blue', size=20, symbol='triangle-up'),
            text=[f"Buy for {trade['price']:.2f} at {trade['time'].strftime('%m-%d %H:%M')}<br>Worth: {(trade['price'] * trade['position'] + trade['balance']):.2f}<br>Position: {trade['position']}" for trade in buys]
        ),
        go.Scatter(
            x=[trade['time'] for trade in sells],
            y=[trade['price'] for trade in sells],
            mode='markers',
            name='Sells',
            marker=dict(color='orange', size=20, symbol='triangle-down'),
            text=[f"Sell for {trade['price']:.2f} at {trade['time'].strftime('%m-%d %H:%M')}<br>Worth: {(trade['price'] * -trade['position'] + trade['balance']):.2f}" for trade in sells]
        )
    ], disable_min_max_vwap=True, price_range=price_range)

for hour in range(len(unique_hours)):
    plot_dual_thrust(hour, k1=0.7, k2=0.7, price_range=[df['Price'].min() - 5, df['Price'].max() + 5])
    # plot_dual_thrust(hour, k1=0.7, k2=0.7)

In [9]:
from itertools import product

# Define reasonable ranges for k1 and k2
k1_values = np.arange(0.1, 1.01, 0.1)
k2_values = np.arange(0.1, 1.01, 0.1)

results_grid = []
initial_balance = 10000

for k1, k2 in product(k1_values, k2_values):
    all_results = {}
    for hour in unique_hours:
        df_minmax_hour = resampled_df[resampled_df['DeliveryHour'] == hour].sort_values('TransactionTime').copy()
        result = backtest_dual_thrust(df_minmax_hour, k1=k1, k2=k2, initial_balance=initial_balance, order_book=all_agg_open_interest)
        all_results[hour] = result
    net_profits = [result['net_profit'] for result in all_results.values()]
    total_profit = np.nansum(net_profits)
    results_grid.append({'k1': k1, 'k2': k2, 'total_profit': total_profit})

In [10]:
k1_vals = np.array([r['k1'] for r in results_grid])
k2_vals = np.array([r['k2'] for r in results_grid])
profits = np.array([r['total_profit'] for r in results_grid])

fig = go.Figure(data=[
    go.Scatter3d(
        x=k1_vals,
        y=k2_vals,
        z=profits,
        mode='markers',
        marker=dict(
            size=5,
            color=profits,
            colorscale='Viridis',
            colorbar=dict(title='Total Profit'),
            opacity=0.8
        ),
        text=[f"k1={k1:.2f}, k2={k2:.2f}, profit={p:.2f}" for k1, k2, p in zip(k1_vals, k2_vals, profits)],
        name='Profit Points'
    )
])

fig.update_layout(
    title='3D Scatter plot of k1, k2 and Total Profit',
    scene=dict(
        xaxis_title='K1 Value',
        yaxis_title='K2 Value',
        zaxis_title='Total Profit'
    )
)

fig.show()